In [101]:
import pandas as pd
import os

In [120]:
# Caminho do arquivo Base_Checkpoint.xlsx 
# arquivo_checkpoint = (r'C:\Users\user\Desktop\Case LongForm\Base_Checkpoint.xlsx')

arquivo_checkpoint = r'C:\Users\user\Desktop\Case LongForm\Base_Checkpoint.xlsx'

In [103]:
#Caminho da pasta onde estão os arquivos do Sistema 1
pasta_sistema1 = (r'C:\Users\user\Desktop\Case LongForm\Sistema1')

In [104]:
print("Arquivos na pasta:", os.listdir(pasta_sistema1))

Arquivos na pasta: ['Base_Sistema1_19-06-2025.xlsx', 'Base_Sistema1_20-06-2025.xlsx']


In [105]:
# Carregando os dados do Sistema 1 (Localizando todos os arquivos do Sistema 1 na pasta e unificando tudo em uma tabela)

lista_tabelas_sistema1 = []
for nome_arquivo in os.listdir(pasta_sistema1):
    # Verificando se o arquivo começa com 'Base_Sistema1' e termina com '.xlsx'
    if nome_arquivo.startswith("Base_Sistema1") and nome_arquivo.endswith(".xlsx"):
        caminho_completo_arquivo = os.path.join(pasta_sistema1, nome_arquivo)
        # Lê o arquivo Excel e guarda na lista
        tabela_temp = pd.read_excel(caminho_completo_arquivo)
        lista_tabelas_sistema1.append(tabela_temp)
print('Verificação dos arquivos concluida')

# Junta todas as tabelas do Sistema 1 em uma só
if lista_tabelas_sistema1:
    tabela_sistema1_completa = pd.concat(lista_tabelas_sistema1, ignore_index=True)
    # Renomeia a coluna 'WB' para 'Numero_WB_Sistema1' para não confundir depois
    tabela_sistema1_completa.rename(columns={"WB": "Numero_WB_Sistema1"}, inplace=True)
else:
    tabela_sistema1_completa = pd.DataFrame()
    print("A lista 'lista_tabelas_sistema1' está vazia.")

Verificação dos arquivos concluida


In [106]:
# Visualizar tabela
print(tabela_sistema1_completa)

     Numero_WB_Sistema1 Long Form Concluído Long Form não concluído devido  \
0            9043848633                 Não                    Not Started   
1            9084986689                 Não                    Not Started   
2            9949996634                 Sim                            NaN   
3            9942099636                 Não                    Not Started   
4            9936433904                 Sim                            NaN   
..                  ...                 ...                            ...   
495          9247773293                 Não                    Not Started   
496          9339090020                 Não                    Not Started   
497          9339969642                 Não                    Not Started   
498          9339292282                 Não                    Not Started   
499          9632989623                 Não                    Not Started   

        Data PU País origem IATA origem IATA destino          .

In [121]:
# Carregando os dados do Checkpoint ('Event' e 'Shipment')

tabela_eventos_checkpoint = pd.read_excel(arquivo_checkpoint, sheet_name="Event")
tabela_remessas_checkpoint = pd.read_excel(arquivo_checkpoint, sheet_name="Shipment")

In [108]:
# Renomeando algumas colunas para facilitar
tabela_eventos_checkpoint.rename(columns={
    "HWB No": "Numero_WB_Evento",
    "Event Cd": "Codigo_do_Evento",
    "Event Dtm": "Data_e_Hora_do_Evento"
}, inplace=True)
tabela_remessas_checkpoint.rename(columns={
    "HWB No": "Numero_WB_Remessa",
    "Last Event Fac": "Ultimo_Local_Evento",
    "Last Event Stn": "Ultima_Estacao_Evento"
}, inplace=True)

print('Dados do Checkpoint carregados.')

Dados do Checkpoint carregados.


In [109]:
print(lista_tabelas_sistema1)  # verifique se não está vazia

[             WB Long Form Concluído Long Form não concluído devido  \
0    9043848633                 Não                    Not Started   
1    9084986689                 Não                    Not Started   
2    9949996634                 Sim                            NaN   
3    9942099636                 Não                    Not Started   
4    9936433904                 Sim                            NaN   
..          ...                 ...                            ...   
333  9632906832                 Não                    Not Started   
334  9632997939                 Não                    Not Started   
335  9632929093                 Não                    Not Started   
336  9632960032                 Não                    Not Started   
337  9689904309                 Não                    Not Started   

        Data PU País origem IATA origem IATA destino          .1     .2  
0    19/06/2025      Mexico         TLC          CPQ  NaN    NaN    NaN  
1    19/06

In [110]:
print(tabela_sistema1_completa.columns)
print(tabela_remessas_checkpoint.columns)

Index(['Numero_WB_Sistema1', 'Long Form Concluído',
       'Long Form não concluído devido', 'Data PU', 'País origem',
       'IATA origem', 'IATA destino', '   ', '   .1', '   .2', 'Unnamed: 7',
       '    '],
      dtype='object')
Index(['Numero_WB_Remessa', 'Orig Ctry', 'Orig Stn', 'Orig Fac', 'Dest Ctry',
       'Dest Stn', 'Dest Fac', 'Prod', 'Piece No', 'Weight', 'Vol Weight',
       'Payer Acct', 'Value', 'Currency', 'Est. USD Value', 'Clock Start',
       'EDD', 'Ultima_Estacao_Evento', 'Ultimo_Local_Evento', 'Last Event Cd',
       'Last Event Dtm', 'Last Event Dtm GMT', '  ', '  .1', '   ', '   .1'],
      dtype='object')


In [111]:
# Juntando todas as informações (Colocando todas as informações juntas em uma tabela.)

# Unindo a tabela do Sistema 1 com as informações de remessa do Checkpoint
tabela_combinada = pd.merge(tabela_sistema1_completa, tabela_remessas_checkpoint,
                            left_on="Numero_WB_Sistema1", right_on="Numero_WB_Remessa", how="left")
# Unindo o que sobrou com as informações de eventos do Checkpoint
tabela_combinada = pd.merge(tabela_combinada, tabela_eventos_checkpoint,
                            left_on="Numero_WB_Sistema1", right_on="Numero_WB_Evento", how="left")

# Renomeandomais algumas colunas para ficar mais claro
tabela_combinada.rename(columns={
    "Long Form Concluído": "Status_Longform_Concluido",
    "Long Form não concluído devido": "Motivo_Longform_Nao_Concluido"
}, inplace=True)

print('Tabelas combinadas com sucesso.')


Tabelas combinadas com sucesso.


In [112]:
print(tabela_combinada.columns)


Index(['Numero_WB_Sistema1', 'Status_Longform_Concluido',
       'Motivo_Longform_Nao_Concluido', 'Data PU', 'País origem',
       'IATA origem', 'IATA destino', '   _x', '   .1_x', '   .2',
       'Unnamed: 7', '    _x', 'Numero_WB_Remessa', 'Orig Ctry', 'Orig Stn',
       'Orig Fac', 'Dest Ctry', 'Dest Stn', 'Dest Fac', 'Prod', 'Piece No',
       'Weight', 'Vol Weight', 'Payer Acct', 'Value', 'Currency',
       'Est. USD Value', 'Clock Start', 'EDD', 'Ultima_Estacao_Evento',
       'Ultimo_Local_Evento', 'Last Event Cd', 'Last Event Dtm',
       'Last Event Dtm GMT', '  _x', '  .1', '   _y', '   .1_y',
       'Numero_WB_Evento', 'Event Stn', 'Event Fac', 'Codigo_do_Evento',
       'Data_e_Hora_do_Evento', 'Event Dtm GMT', '    _y', '  _y', '   ', ' ',
       ' .1', ' .2', ' .3', ' .4'],
      dtype='object')


In [113]:
# Limpeza e Seleção de Colunas

# Removendo colunas que estão completamente vazias (só têm valores nulos)

tabela_combinada.dropna(axis=1, how='all', inplace=True)

# Removendo colunas sem nome ou com nomes estranhos

colunas_para_remover_por_nome = [col for col in tabela_combinada.columns if str(col).startswith("Unnamed:") or str(col).startswith(" _") or str(col).startswith(" .")]
tabela_combinada.drop(columns=colunas_para_remover_por_nome, inplace=True, errors='ignore')

# Selecionar apenas as colunas que serão utilizadas na análise

colunas_importantes = [
    "Numero_WB_Sistema1",
    "Status_Longform_Concluido",
    "Motivo_Longform_Nao_Concluido",
    "Codigo_do_Evento",
    "Data_e_Hora_do_Evento",
    "Ultimo_Local_Evento",
    "Ultima_Estacao_Evento",
    "Numero_WB_Remessa" # Manter para verificar WBs sem correspondência
]

# Garantir que todas as colunas importantes existem antes de selecionar

existing_columns = [col for col in colunas_importantes if col in tabela_combinada.columns]
tabela_combinada = tabela_combinada[existing_columns]

print(print(tabela_combinada.columns))

Index(['Numero_WB_Sistema1', 'Status_Longform_Concluido',
       'Motivo_Longform_Nao_Concluido', 'Codigo_do_Evento',
       'Data_e_Hora_do_Evento', 'Ultimo_Local_Evento', 'Ultima_Estacao_Evento',
       'Numero_WB_Remessa'],
      dtype='object')
None


In [114]:
# Aplicando as Regras de Negócio (filtrar o que interessa)

# REGRA A: A remessa não pode estar concluída e o motivo deve ser 'Not Started' (Isso significa que ela ainda pode ser processada para Longform.)

tabela_elegivel = tabela_combinada[
    (tabela_combinada["Status_Longform_Concluido"] == "Não") &
    (tabela_combinada["Motivo_Longform_Nao_Concluido"] == "Not Started")
].copy()

In [115]:
# REGRA B: A remessa precisa ter os eventos 'PU' (Pickup) e 'RW' (Received at Warehouse) / (Verificando quais WBs têm esses eventos.)

wbs_com_pu = tabela_elegivel[tabela_elegivel["Codigo_do_Evento"] == "PU"]["Numero_WB_Sistema1"].unique()
wbs_com_rw = tabela_elegivel[tabela_elegivel["Codigo_do_Evento"] == "RW"]["Numero_WB_Sistema1"].unique()

# Separando somente as WBs que têm AMBOS os eventos
wbs_com_ambos_pu_rw = list(set(wbs_com_pu) & set(wbs_com_rw))
tabela_elegivel = tabela_elegivel[tabela_elegivel["Numero_WB_Sistema1"].isin(wbs_com_ambos_pu_rw)]

In [116]:
# REGRA C: Excluir remessas que já foram finalizadas (Se o último evento for 'CS', 'RT' ou 'OK', a remessa já terminou e não precisa de Longform.)

# Garantimos que a coluna de data é realmente uma data
tabela_elegivel["Data_e_Hora_do_Evento"] = pd.to_datetime(tabela_elegivel["Data_e_Hora_do_Evento"])

# Encontrando o evento mais recente para cada WB
ultimo_evento_por_wb = tabela_elegivel.sort_values("Data_e_Hora_do_Evento", ascending=False).drop_duplicates("Numero_WB_Sistema1")

# Códigos de evento que indicam que a remessa está finalizada
codigos_finalizados = ["CS", "RT", "OK"]

# Encontrando as WBs que têm um desses códigos como último evento
wbs_para_remover = ultimo_evento_por_wb[
    ultimo_evento_por_wb["Codigo_do_Evento"].isin(codigos_finalizados)
]["Numero_WB_Sistema1"]

# Removendo essas WBs da lista de elegíveis
tabela_final_elegivel = tabela_elegivel[~tabela_elegivel["Numero_WB_Sistema1"].isin(wbs_para_remover)]

In [117]:
# Identificar Gateways

# Garantindo que tabela_final_elegivel seja uma cópia para evitar warnings
tabela_final_elegivel = tabela_final_elegivel.copy()

# Adicionando informações sobre o Gateway para análise.

tabela_final_elegivel["Gateway_Identificado"] = tabela_final_elegivel["Ultimo_Local_Evento"].apply(
    lambda x: x if pd.notna(x) and "GTW" in str(x) else None
)
tabela_final_elegivel["Estacao_Gateway"] = tabela_final_elegivel["Ultima_Estacao_Evento"]

In [118]:
# Salvando os Resultados Finais

# Salvando a tabela com as remessas elegíveis em um arquivo CSV.

tabela_final_elegivel.to_csv("filtered_data.csv", index=False)
print("   Arquivo 'filtered_data.csv' criado.")

# Identificando as WBs do Sistema 1 que não encontraram correspondência

# Se a coluna 'Numero_WB_Remessa' estiver vazia, significa que não achou correspondência

wbs_sem_correspondencia = tabela_combinada[
    tabela_combinada["Numero_WB_Remessa"].isnull()
]["Numero_WB_Sistema1"].unique()

# Salvando essa lista em outro arquivo CSV
tabela_wbs_sem_correspondencia = pd.DataFrame(wbs_sem_correspondencia, columns=["WB_Para_Extracao_Manual"])
tabela_wbs_sem_correspondencia.to_csv("wbs_sem_correspondencia.csv", index=False)
print("   Arquivo 'wbs_sem_correspondencia.csv' criado.")

   Arquivo 'filtered_data.csv' criado.
   Arquivo 'wbs_sem_correspondencia.csv' criado.


In [119]:
# Gerando um Resumo

# Total de WBs que podem ter o Longform
total_wbs_elegiveis = tabela_final_elegivel["Numero_WB_Sistema1"].nunique()
print(f"\nTotal de WBs elegíveis para Longform: {total_wbs_elegiveis}")

# Contagem por Gateway prioritário
gateways_prioritarios = ["MIA", "MAD", "CVG", "LEJ", "BCN"]
print("Contagem de WBs por Gateway Prioritário:")
for gtw in gateways_prioritarios:
    contagem = tabela_final_elegivel[
        tabela_final_elegivel["Estacao_Gateway"] == gtw
    ]["Numero_WB_Sistema1"].nunique()
    print(f"  GTW {gtw}: {contagem} WBs")

# Quantas WBs precisam de extração manual
num_wbs_manual = tabela_wbs_sem_correspondencia.shape[0]
print(f"\nNúmero de WBs para extração manual: {num_wbs_manual}")
if num_wbs_manual > 0:
    print("Lista de WBs para extração manual (ver arquivo 'wbs_sem_correspondencia.csv'):")
    print(tabela_wbs_sem_correspondencia)


Total de WBs elegíveis para Longform: 388
Contagem de WBs por Gateway Prioritário:
  GTW MIA: 24 WBs
  GTW MAD: 18 WBs
  GTW CVG: 4 WBs
  GTW LEJ: 1 WBs
  GTW BCN: 43 WBs

Número de WBs para extração manual: 0
